In [1]:
# Importing necessary models
import warnings
warnings.filterwarnings('ignore')

import smtplib
import pandas as pd
import numpy as np
import datetime as dt
import pandas.stats.moments as st
import sqlite3 as sql
import time
%matplotlib inline
import os
main_dir = os.getcwd()

from option_slam_earnings_db import *
os.chdir('C:\\Users\\Fang\\Desktop\\Python Trading\\Trading\\Trading\\Modules\\DataCollection')

from yahoo_query import *
from reuters_query import reuters_query
from alphaquery import alphaquery

# os.chdir('C:\\Users\\Fang\\Desktop\\Python Trading\\Trading\\Trading\\Modules\\Earnings')
# from yahoo_earnings import *

os.chdir('C:\\Users\\Fang\\Desktop\\Python Trading\\Trading\\Data\\Stock Universe')
cad_stocks = pd.read_csv('cad_names.csv')['Symbol'].tolist()
usd_stocks = pd.read_csv('us_names.csv')['Symbol'].tolist()

os.chdir('C:\\Users\\Fang\\Desktop\\Python Trading\\Trading\\Data\\DBs')

In [ ]:
conn_yahoo = sql.connect('yahoo.db')
conn_reuters = sql.connect('reuters.db')
conn_optionslam = sql.connect('optionslam.db')
conn_alphaquery = sql.connect('alphaquery.db')
#conn.close()
# Functions to create tables

def create_tables(db_name = 'nba'):
    conn = sqlite3.connect('{}.db'.format(db_name))
    cur = conn.cursor()
    
    # Creating Players Table
    cur.execute('DROP TABLE IF EXISTS players ')
    cur.execute('CREATE TABLE players (player_id INTEGER PRIMARY KEY, firstname TEXT, lastname TEXT, player_link TEXT, \
                                       college_id INTEGER, team_id INTEGER, total_points INTEGER)')
    
    # Creating Colleges Table
    cur.execute('DROP TABLE IF EXISTS colleges ')
    cur.execute('CREATE TABLE colleges (college_id INTEGER PRIMARY KEY, college_name TEXT)')
    
    # Creating Teams Table
    cur.execute('DROP TABLE IF EXISTS teams ')
    cur.execute('CREATE TABLE teams (team_id INTEGER PRIMARY KEY, team_name TEXT)')
    
    conn.close()
    return

def insert_player(player, db_name = 'nba'):
    conn = sqlite3.connect('{}.db'.format(db_name))
    cur = conn.cursor()
    
    cur.execute('INSERT INTO players (player_id, firstname, lastname, player_link, \
                                      college_id, team_id, total_points) VALUES \
                                     ( ?, ?, ?, ?, ?, ?, ? )', 
    ( player['player_id'], player['firstname'], player['lastname'], 
      player['player_link'], player['college'], player['team'], player['total_points'] ) )

    conn.commit()
    conn.close()
    return

def insert_teams(teams, db_name = 'nba'):
    conn = sqlite3.connect('{}.db'.format(db_name))
    cur = conn.cursor()
    
    for team_name, team_id in teams.items():
        cur.execute('INSERT INTO teams (team_id, team_name) VALUES ( ?, ?)', 
                    ( team_id, team_name ) )

    conn.commit()
    conn.close()
    return

def insert_colleges(colleges, db_name = 'nba'):
    conn = sqlite3.connect('{}.db'.format(db_name))
    cur = conn.cursor()
    
    for college_name, college_id in colleges.items():
        cur.execute('INSERT INTO colleges (college_id, college_name) VALUES ( ?, ?)', 
                    ( college_id, college_name ) )

    conn.commit()
    conn.close()
    return

In [17]:
ticker = 'AMD'

start_time = time.time()

start_date = dt.datetime.now() - dt.timedelta(days = 900)
curr_optionslam = earnings_report(ticker, corr_window = 60)

curr_alphaquery = alphaquery(ticker)
curr_yahoo = yahoo_query(ticker, start_date)
curr_yahoo.full_info_query()
curr_reuters = reuters_query(ticker)
curr_yahoo.hist_prices_query()


print("--- %s seconds ---" % (time.time() - start_time))

--- 8.466244220733643 seconds ---


In [18]:
# Optionslam Data
curr_optionslam

,AMD,AMD_closeToOpen,Technology Beta,MarketBeta,AMD_52WeekReturn,SPY_52WeekReturn,XLK_52WeekReturn
2016-07-21,AC,0.097701,0.420021,0.405454,2.005556,0.025170,0.055786
2016-10-20,AC,-0.066092,0.455605,0.430224,2.351485,0.054995,0.133270
2017-01-31,AC,0.051109,0.510242,0.318854,3.822727,0.174633,0.220417
2017-05-01,AC,-0.138767,0.362303,0.319916,2.684211,0.147650,0.279529
2017-07-25,AC,0.072289,0.396462,0.231462,1.424657,0.136163,0.257037
2017-10-24,AC,-0.091228,0.512109,0.443200,1.162577,0.196888,0.279403
2018-01-30,AC,0.028749,0.539816,0.309255,0.248360,0.243307,0.351202
2018-04-25,AC,0.119464,0.625314,0.607313,-0.231531,0.108825,0.204833
2018-07-25,AC,0.069159,0.629235,0.600249,0.143362,0.140953,0.272696
2018-10-24,AC,-0.213690,0.386937,0.302109,0.779433,0.068330,0.156389


In [19]:
# Alphaquery Data
curr_alphaquery.vol_df

,10-Day,20-Day,30-Day,60-Day,90-Day,120-Day,150-Day,180-Day,Underlying
Field,,,,,,,,,
Historical Volatility (Close-to-Close),1.4595,1.2517,1.0779,0.9025,0.8162,0.7634,0.7113,0.6693,AMD
Historical Volatility (Parkinson),0.9366,0.8646,0.7868,0.7207,0.6662,0.6146,0.5932,0.5569,AMD
Implied Volatility (Calls),0.7647,0.7185,0.7079,0.6616,0.6657,0.6684,0.6476,0.6352,AMD
Implied Volatility (Puts),0.7645,0.7188,0.7073,0.6631,0.6650,0.6675,0.6487,0.6376,AMD
Implied Volatility (Mean),0.7646,0.7186,0.7076,0.6624,0.6654,0.6680,0.6482,0.6364,AMD
Put-Call Implied Volatility Ratio,0.9997,1.0004,0.9992,1.0023,0.9989,0.9987,1.0017,1.0038,AMD
Implied Volatility Skew,0.0334,0.0345,0.0383,0.0325,0.0261,0.0231,0.0195,0.0173,AMD
Put-Call Ratio (Volume),0.6476,0.3209,0.3063,0.5697,0.3753,0.4637,0.8424,0.9971,AMD
Put-Call Ratio (Open Interest),0.5014,0.5776,0.6668,0.9616,1.1152,1.1478,1.0861,1.0426,AMD


In [24]:
# Yahoo Data

curr_yahoo.hist_prices

curr_yahoo.earnings_annual

curr_yahoo.earnings_quarterly 

curr_yahoo.profile 

curr_yahoo.executives 

curr_yahoo.cashFlowStatementAnnual 

curr_yahoo.cashFlowStatementQuarter 

curr_yahoo.institutionOwners 

curr_yahoo.majorHolderInfo 

curr_yahoo.recommendationTrend 

curr_yahoo.keyStats 

curr_yahoo.purchaseActivity 

curr_yahoo.insiderTxns 

curr_yahoo.incomeStatementAnnual 

curr_yahoo.incomeStatementQuarter 

curr_yahoo.balanceSheetAnnual 

curr_yahoo.balanceSheetQuarter 

curr_yahoo.fundOwnership 

curr_yahoo.insiderHolders 

try:
    curr_yahoo.currEarnings 
except:
    None

try:
    curr_yahoo.dividends 
except:
    None

curr_yahoo.earningEsts 

curr_yahoo.finData 

,currentPrice,currentRatio,debtToEquity,earningsGrowth,ebitda,ebitdaMargins,financialCurrency,freeCashflow,grossMargins,grossProfits,...,revenueGrowth,revenuePerShare,targetHighPrice,targetLowPrice,targetMeanPrice,targetMedianPrice,totalCash,totalCashPerShare,totalDebt,totalRevenue
AMD,20.23,1.659,158.476,NaN,628000000,0.09808,USD,150375008,0.36046,1823000000,...,0.526,6.628,40,7,23.87,25,983000000,1.008,1392999936,6402999808


In [29]:
curr_yahoo.cashFlowStatementAnnual 

,capitalExpenditures,changeInCash,changeToAccountReceivables,changeToInventory,changeToLiabilities,changeToNetincome,changeToOperatingActivities,depreciation,investments,issuanceOfStock,netBorrowings,netIncome,otherCashflowsFromFinancingActivities,otherCashflowsFromInvestingActivities,totalCashFromFinancingActivities,totalCashFromOperatingActivities,totalCashflowsFromInvestingActivities
2017-12-30,-113000000,-79000000,-89000000,12000000,-75000000,109000000,-112000000,144000000,1000000,20000000,-40000000,43000000,-13000000,-2000000,-33000000,68000000,-114000000
2016-12-31,-77000000,479000000,222000000,-73000000,58000000,253000000,-27000000,133000000,342000000,687000000,-561000000,-497000000,-4000000,2000000,122000000,90000000,267000000
2015-12-26,-96000000,-20000000,280000000,-11000000,-156000000,143000000,-27000000,167000000,235000000,5000000,56000000,-660000000,-2000000,2000000,59000000,-226000000,147000000
2014-12-27,-95000000,-64000000,7000000,199000000,-231000000,393000000,-266000000,190000000,83000000,4000000,40000000,-403000000,2000000,2000000,46000000,-98000000,-12000000


In [25]:
# Reuters Data

curr_reuters.overall_df

# Financial Ratio Tables
curr_reuters.revenues_eps_df

curr_reuters.sales_ests
curr_reuters.earnings_ests
curr_reuters.LTgrowth_ests

curr_reuters.valuations
curr_reuters.dividends
curr_reuters.growthrate
curr_reuters.finstrength
curr_reuters.profitability
curr_reuters.efficiency
curr_reuters.management

curr_reuters.growth_summary
curr_reuters.performance_summary
curr_reuters.institution_holdings

# Analyst Tables
curr_reuters.recommendations
curr_reuters.analyst_recs

curr_reuters.sales_analysis
curr_reuters.earnings_analysis
curr_reuters.LTgrowth_analysis
curr_reuters.sales_surprises
curr_reuters.earnings_surprises
curr_reuters.sales_trend
curr_reuters.earnings_trend

curr_reuters.revenue_revisions
curr_reuters.earnings_revisions 

,Down Last4Weeks,Down LastWeek,NumberOfRevisions,Up Last4Weeks,Up LastWeek,Underlying
0,17.0,1.0,Quarter Ending Dec-18,5.0,0.0,AMD
1,17.0,1.0,Quarter Ending Mar-19,2.0,0.0,AMD
2,17.0,2.0,Year Ending Dec-18,5.0,0.0,AMD
3,17.0,1.0,Year Ending Dec-19,5.0,0.0,AMD
